# grocery_sales_forecasting_storewise

This baseline model is non-parametric. This model is simply average of history sale (from 2013 to 2016 of the same day.)

In [1]:
# Import libraries necessary for this project
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit, train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import preprocessing
# from sklearn_pandas import DataFrameMapper
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.pipeline import FeatureUnion

from xgboost import XGBRegressor

from IPython.display import display
import matplotlib.pyplot as plt
import datetime as dt
from time import time
import sys
from collections import OrderedDict
from fbprophet import Prophet
import calendar

types = {'id': 'int32', 'item_nbr': 'int32', 'store_nbr': 'int8', 'onpromotion': bool}

%matplotlib inline

## import data

There are 19 items appear in training data, but not in testing data; 68 items appear only testing data, not in training data.

In [2]:
# from sklearn.preprocessing import OneHotEncoder
def pre_process(feature_raw):

    items = pd.read_csv("input/items.csv", dtype={'perishable': np.dtype('int8')})
    feature_raw= pd.merge(feature_raw,items, right_on='item_nbr',left_on='item_nbr',how='left')

    feature_raw['onpromotion'].fillna(False, inplace = True)
    feature_raw['onpromotion'] = feature_raw['onpromotion'].map({False : 0, True : 1})
    feature_raw['onpromotion'] = feature_raw['onpromotion'].astype('int8')

    if feature_raw.isnull().values.any():
        print("There is NaN before one-hot-encoding!")
    else:
        print("There is no NaN before one-hot-encoding")    

    feature_one_hot_coded = pd.get_dummies(feature_raw, columns = ['item_nbr', 'family', 'class'])

    if feature_one_hot_coded.isnull().values.any():
        print("There is NaN after one-hot-encoding!")
    else:
        print("There is no NaN after one-hot-encoding")
        
    return feature_one_hot_coded

In [3]:
def nan_alarm(df):
    if df.isnull().values.any():
        print("There is NaN")
    else:
        print("There is no NaN")
    return

def data_preprocess(store):
    ### Load training data and testing data, and put them together for encoding ###
    filename_train = 'train_store_' + str(store) + '.csv'
    train_data_raw = pd.read_csv('input_each_store/' + filename_train, usecols = [1,2,3,4,5],
                                 parse_dates=['date'], dtype=types, #skiprows=range(1,200000),
                                 infer_datetime_format = True,
                                 converters={'unit_sales':lambda u: float(u) if float(u)>0 else 0},)
    print("Raw training data has {} samples with {} features each.".format(*train_data_raw.shape))
    train_data_raw.dropna(inplace=True)
    
    filename_test = 'test_store_' + str(store) + '.csv'
    test_data_raw = pd.read_csv('input_each_store/' + filename_test, usecols=[1,2,3,4], parse_dates=['date'], dtype=types)
    print("Public testing data has {} samples with {} features each.".format(*test_data_raw.shape))
    test_data_raw.dropna(inplace=True)

    data_all = pd.concat([train_data_raw.drop('unit_sales', axis=1), test_data_raw])

    ### Extend training data information ###
    items = pd.read_csv("input/items.csv", dtype={'perishable': np.dtype('int8')})
    items.dropna(inplace=True)
    data_all= pd.merge(data_all,items, right_on='item_nbr',left_on='item_nbr',how='left')
    
    transactions = pd.read_csv("input/transactions.csv", parse_dates=['date'])
    transcations_store = transactions[transactions['store_nbr']==store].dropna()
    transaction_pred = predict_transactions(store)
    transactins_all = pd.concat([transcations_store.drop('store_nbr', axis=1), transaction_pred])    
    data_all = pd.merge(data_all, transactins_all, left_on=['date'], right_on=['date'], how='left')    
    data_all.drop(['store_nbr'], axis = 1, inplace = True)

    ### Encoding ###
#     #One_hot encoding
#     feature_one_hot_coded = pd.get_dummies(data_all, columns = ['item_nbr', 'family', 'class'])
    #Binary encoding
    categorical_items = ['item_nbr', 'family', 'class']
    for col in categorical_items:
        binaryEncode = preprocessing.LabelBinarizer().fit_transform(data_all[col]).astype(np.int8)
        df_temp = pd.DataFrame(binaryEncode)
        df_temp = df_temp.add_prefix(col)
        data_all = pd.concat([data_all, df_temp], axis=1)
        del data_all[col]
    data_all['onpromotion'] = preprocessing.Binarizer().fit_transform(data_all['onpromotion'].values.reshape(-1,1)).astype(np.int8)
    data_all['perishable'] = data_all['perishable'].astype(np.int8)
    data_all['transactions'] = preprocessing.MinMaxScaler().fit_transform(data_all['transactions'].values.reshape(-1,1))
        
    
    ### Separate training out to train PCA ####
    train_data = data_all[:train_data_raw.shape[0]].copy()
    train_target = train_data_raw['unit_sales'].apply(np.log1p).values
    ### Feature Selection: PCA ####
    pca = PCA(n_components = 500).fit(train_data.drop('date', axis=1), train_target)
    importances = pca.explained_variance_ratio_
#     feature_plot(importances)
#     selection = SelectKBest(f_regression,k=1)
#     combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)]).fit(train_data, train_target)
    features = pca.transform(data_all.drop('date', axis=1))
    
    ### Concat extra column and target column to training data for returning as a whole ###
    columns_keeped = ['date', 'onpromotion', 'perishable', 'transactions']
    features_processed = pd.DataFrame(features).add_prefix('pca')
    features_processed[columns_keeped] = data_all[columns_keeped]
    
    train_processed = features_processed[:train_data_raw.shape[0]].copy()
    train_processed['unit_sales'] = train_target
    test_processed = features_processed[train_data_raw.shape[0]:].copy()
    
    return [train_processed, test_processed]

Evaluation metric: Normalized Weighted Root Mean Squared Logarithmic Error (NWRMSLE)

In [4]:
def store_loc_search(store):
    location = {}
    
    stores_raw = pd.read_csv('input/stores.csv')
    location['city'] = stores_raw.loc[stores_raw['store_nbr']==store, 'city'].values[0]
    location['state'] = stores_raw.loc[stores_raw['store_nbr']==store, 'state'].values[0]
    return location

def holidays(store):
    #holidays_events happens at the region of a store
    city,state = store_loc_search(store)
    holidays_events_raw = pd.read_csv('input/holidays_events.csv', parse_dates=['date'])
    mask = (holidays_events_raw['date'] >= '2015-01-01') & (holidays_events_raw['transferred'] == False)&(
        ((holidays_events_raw['locale']=='Local')&(holidays_events_raw['locale_name']==city))|
        ((holidays_events_raw['locale']=='Reginal')&(holidays_events_raw['locale_name']==state))|
        (holidays_events_raw['locale']=='National')
    )
    holidays_events = pd.DataFrame({
        'holiday':holidays_events_raw[mask]['type'],
        'ds':holidays_events_raw[mask]['date'],
        'lower_window':0,
        'upper_window':1,
    })
    
    #payoff days: Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month.
    payoff_dates = []
    for year in range(2015, 2018):
        for month in range(1,13):
            payoff_dates.append(str(year)+'-'+str(month)+'-15')
            payoff_dates.append(str(year)+'-'+str(month)+'-'+str(calendar.monthrange(year,month)[1]))
    payoffs = pd.DataFrame({
      'holiday': 'playoff',
      'ds': pd.to_datetime(payoff_dates),
      'lower_window': 0,
      'upper_window': 1,
    })
    
    earthquake = pd.DataFrame({
      'holiday': 'earthquake',
      'ds': pd.to_datetime(['2016-04-16']),
      'lower_window': 0,
      'upper_window': 14,
    })
    
    #Put holiday_events and payoff_dates together
    holidays = pd.concat((holidays_events, payoffs, earthquake)).fillna(0).sort_values(['ds'])
    return holidays

def transactions(store):
    transactions_raw = pd.read_csv('input/transactions.csv', parse_dates = ['date'])
    transactions = transactions_raw.loc[transactions_raw['store_nbr']==store].copy()
    transactions.drop(['store_nbr'], axis=1, inplace=True)
    transactions = transactions.rename(columns={'date': 'ds','transactions': 'y'})
    return transactions

def predict_transactions(store):
    phophet_model = Prophet(holidays=holidays(store))
#     phophet_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    transactions_store = transactions(store)
    phophet_model.fit(transactions_store)
    future = phophet_model.make_future_dataframe(periods=16, freq='d')
    forecast = phophet_model.predict(future)
    transaction_pred = forecast.loc[forecast['ds']>='2017-08-16',['ds', 'yhat']]
    transaction_pred = transaction_pred.rename(columns={'ds': 'date', 'yhat': 'transactions'})
    return transaction_pred

In [5]:
from sklearn.metrics import make_scorer, mean_squared_error
def nwrmsle(ground_truth, predictions, w):
    return mean_squared_error(ground_truth, predictions, sample_weight=w)**0.5

nwrmsle_scorer = make_scorer(score_func=mean_squared_error, greater_is_better=False)

In [6]:
def train_predict(learner, data):
    result = {}
    y_train = data['unit_sales']; X_train = data.drop(['date','unit_sales'], axis=1)
    w_train = data['perishable'].apply(lambda x: 1.25 if x else 1).values
#     nwrmsle_scorer = make_scorer(score_func=mean_squared_error, greater_is_better=False, sample_weight=w_train)

    params = {'max_depth':[25], 'n_estimators':[10]}
#     params_test = {'max_depth':[25], 'n_estimators':[40]}
    grid_obj = GridSearchCV(learner, param_grid = params, scoring = 'neg_mean_squared_error')
    grid_fit = grid_obj.fit(X_train, y_train, sample_weight=w_train)
    
    result['learner'] = grid_fit.best_estimator_

    return result

In [ ]:
from xgboost.sklearn import XGBRegressor  
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV

def train_predict_RS(learner, data):
    result = {}
    y_train = data['unit_sales']; X_train = data.drop(['date','unit_sales'], axis=1)
    w_train = data['perishable'].apply(lambda x: 1.25 if x else 1).values
#     nwrmsle_scorer = make_scorer(score_func=mean_squared_error, greater_is_better=False, sample_weight=w_train)

    one_to_left = st.beta(10, 1)  
    from_zero_positive = st.expon(0, 50)

    params = {  
        "n_estimators": st.randint(3, 40),
        "max_depth": st.randint(3, 40),
        "learning_rate": st.uniform(0.05, 0.4),
        "colsample_bytree": one_to_left,
        "subsample": one_to_left,
        "gamma": st.uniform(0, 10),
        'reg_alpha': from_zero_positive,
        "min_child_weight": from_zero_positive,
    }

    gs = RandomizedSearchCV(xgbreg, params, n_jobs=1, scoring = 'neg_mean_squared_error')  
    gs.fit(X_train, y_train, sample_weight=w_train)  
    
    result['learner'] = gs.best_model_ 

    return result

## DecisionTreeRegressor

In [ ]:
def predict_submission(model, test_processed, store):
    
    predictions = model.predict(test_processed.drop('date', axis=1))
    
    filename_test = 'test_store_' + str(store) + '.csv'
    test_data_raw = pd.read_csv('input_each_store/' + filename_test, parse_dates=['date'], dtype=types)
    test_data_raw['unit_sales']= [np.expm1(p) for p in predictions]
    display(test_data_raw.head(5))
#     test_store.fillna(0, inplace=True)
    return test_data_raw[['id', 'unit_sales']].set_index(['id'])

In [ ]:
results = {}
store_list = range(1,55)
predictions = pd.DataFrame(0, index = range(125497040, 128867504), columns = ['unit_sales'])
predictions.index.name = 'id'

# regressor_A = DecisionTreeRegressor(random_state = 36, max_depth = 50)
# regressor_B = linear_model.LinearRegression()
# params_gbr = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split':2, 'learning_rate': 0.01}
# regressor_C = GradientBoostingRegressor(**params_gbr)
# params_xgb = {'max_depth':50, 'n_estimators':100, 'objective':'reg:linear'}
# regressor_D = XGBRegressor(**params_xgb)

regressor = XGBRegressor(objective='reg:linear', nthreads=-1)
reg_name = regressor.__class__.__name__

file_submission_temp = reg_name + dt.datetime.today().strftime("%Y-%m-%d") + '_temp.csv'
file_submission = reg_name + dt.datetime.today().strftime("%Y-%m-%d") + '.csv'

for store in store_list:
    print("------------------------------")
    start = time()
    print("Start Store {}:".format(store))
    train_processed, test_processed = data_preprocess(store)
    print("Processed training set has {} samples with {} variables.".format(*train_processed.shape))
    
    
    results[store]= train_predict(regressor, train_processed)
    prediction_store = predict_submission(results[store]['learner'], test_processed, store)
    
    predictions.loc[prediction_store.index] = prediction_store 
    if not os.path.isfile(file_submission_temp):
        prediction_store.to_csv(file_submission_temp, float_format='%.3f')
    else:
        prediction_store.to_csv(file_submission_temp, mode = 'a', header = False, float_format='%.3f')
    print("Store {} is complete in {:.2f} min.".format(store, (time()-start)/60))

predictions.to_csv(file_submission, float_format='%.3f')

------------------------------
Start Store 1:
Raw training data has 208848 samples with 5 features each.
Public testing data has 62416 samples with 4 features each.


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Processed training set has 208848 samples with 505 variables.


,id,date,store_nbr,item_nbr,onpromotion,unit_sales
0,125497040,2017-08-16,1,96995,False,1.120579
1,125497041,2017-08-16,1,99197,False,1.163123
2,125497042,2017-08-16,1,103501,False,1.607258
3,125497043,2017-08-16,1,103520,False,1.715637
4,125497044,2017-08-16,1,103665,False,2.401586


Store 1 is complete in 26.94 min.
------------------------------
Start Store 2:
Raw training data has 243505 samples with 5 features each.
Public testing data has 62416 samples with 4 features each.


...

Processed training set has 212280 samples with 505 variables.


,id,date,store_nbr,item_nbr,onpromotion,unit_sales
0,125699892,2017-08-16,53,96995,False,1.102859
1,125699893,2017-08-16,53,99197,False,1.139604
2,125699894,2017-08-16,53,103501,False,1.404719
3,125699895,2017-08-16,53,103520,False,1.158220
4,125699896,2017-08-16,53,103665,False,1.155968


Store 53 is complete in 27.60 min.
------------------------------
Start Store 54:
Raw training data has 139988 samples with 5 features each.
Public testing data has 62416 samples with 4 features each.


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Processed training set has 139988 samples with 505 variables.


In [ ]:
filename_result = reg_name+'_result.sav'
pickle.dump(results, open(filename_result, 'wb'))